In [1]:
import os
import pandas as pd
import numpy as np

from tqdm import tqdm

from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split

import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPool2D
from keras.utils import to_categorical
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import to_categorical

from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

import cv2
import os

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
labels = ["0","1","2","3","4"]
img_size = 128
def get_data(data_dir):
    data = [] 
    for label in labels: 
        path = os.path.join(data_dir, label)
        class_num = labels.index(label)
        for img in tqdm(os.listdir(path)):
            try:
                img_arr = cv2.imread(os.path.join(path, img))[...,::-1]
                resized_arr = cv2.resize(img_arr, (img_size, img_size))
                data.append([resized_arr, class_num])
            except Exception as e:
                print(e)
    return np.array(data)

In [4]:
images_data = get_data("gdrive/My Drive/knee_data/images")

100%|██████████| 295/295 [00:01<00:00, 211.67it/s]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  from ipykernel import kernelapp as app


In [5]:
X = []
y = []
for feature, label in images_data:
  X.append(feature)
  y.append(label)

In [6]:
X = np.array(X) / 255

In [7]:
X.reshape(-1, img_size, img_size, 1)
y = np.array(y)

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=13)

In [9]:
X_train.shape

(7339, 128, 128, 3)

In [10]:
X_test.shape

(2447, 128, 128, 3)

In [11]:
datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range = 30,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.2, # Randomly zoom image 
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip = True,  # randomly flip images
        vertical_flip=False)  # randomly flip images


datagen.fit(X_train)

In [12]:
model = Sequential()
model.add(Conv2D(32,3,padding="same", activation="relu", input_shape=(128,128,3)))
model.add(MaxPool2D())

model.add(Conv2D(32, 3, padding="same", activation="relu"))
model.add(MaxPool2D())

model.add(Conv2D(32, 3, padding="same", activation="elu"))
model.add(MaxPool2D())

model.add(Conv2D(64, 3, padding="same", activation="elu"))
model.add(MaxPool2D())
model.add(Dropout(0.4))

model.add(Flatten())
model.add(Dense(64,activation="relu"))
model.add(Dense(5, activation="softmax"))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 128, 128, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 64, 64, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 64, 64, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 16, 16, 64)        1

In [13]:
model.compile(optimizer = "adam" , loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True) , metrics = ['accuracy'])

In [14]:
early_stop = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
mcp_save = ModelCheckpoint('gdrive/My Drive/knee_data/image/callback_model.h5', save_best_only=True, monitor='val_loss', mode='min')

In [15]:
history = model.fit(X_train, y_train,
                   validation_data=(X_test, y_test),
                   epochs=40, batch_size=32,
                   verbose=2, callbacks=[early_stop, mcp_save])

Epoch 1/40
230/230 - 135s - loss: 1.4197 - accuracy: 0.3795 - val_loss: 1.4122 - val_accuracy: 0.3919
Epoch 2/40
230/230 - 131s - loss: 1.4060 - accuracy: 0.3834 - val_loss: 1.4033 - val_accuracy: 0.3952
Epoch 3/40
230/230 - 131s - loss: 1.3982 - accuracy: 0.3905 - val_loss: 1.3969 - val_accuracy: 0.3952
Epoch 4/40
230/230 - 130s - loss: 1.3836 - accuracy: 0.4081 - val_loss: 1.3804 - val_accuracy: 0.4054
Epoch 5/40
230/230 - 130s - loss: 1.3700 - accuracy: 0.4137 - val_loss: 1.3407 - val_accuracy: 0.4205
Epoch 6/40
230/230 - 130s - loss: 1.3448 - accuracy: 0.4180 - val_loss: 1.2928 - val_accuracy: 0.4377
Epoch 7/40
230/230 - 130s - loss: 1.2421 - accuracy: 0.4596 - val_loss: 1.2362 - val_accuracy: 0.4659
Epoch 8/40
230/230 - 130s - loss: 1.1732 - accuracy: 0.4886 - val_loss: 1.1191 - val_accuracy: 0.5149
Epoch 9/40
230/230 - 130s - loss: 1.1218 - accuracy: 0.5123 - val_loss: 1.0900 - val_accuracy: 0.5268
Epoch 10/40
230/230 - 130s - loss: 1.0716 - accuracy: 0.5367 - val_loss: 1.0892 - 